In [57]:
# %%writefile xxx.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import numpy as np
import pandas as pd

import QUANTAXIS as QA

import tools.Sample_Tools as smpl
from base.JuUnits import excute_for_multidates

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


In [ ]:
# constance
st ='2016-01-04'
en ='2020-12-31'
fr=QA.FREQUENCE.FIVE_MIN

In [190]:
min5 = smpl.get_data(['000001'], start='2016-01-04', end='2020-12-31', gap=None, freq=QA.FREQUENCE.FIVE_MIN).data
ret5 = min5.close/min5.close.shift(1) - 1
ret5_un = ret5.reset_index()
ret5_un[ret5.name].groupby(ret5_un.datetime.dt.to_period("D"),group_keys=False).apply(lambda x: x.std())
 

datetime
2016-02-22    0.002107
2016-02-23    0.001590
2016-02-24    0.001693
2016-02-25    0.002284
2016-02-26    0.002493
                ...   
2020-12-25    0.002686
2020-12-28    0.003126
2020-12-29    0.003592
2020-12-30    0.004397
2020-12-31    0.003208
Freq: D, Name: close, Length: 1188, dtype: float64

In [185]:
min5 = smpl.get_data(['000002'], start='2017-01-03', end='2017-12-31', gap=None, freq=QA.FREQUENCE.FIVE_MIN).data
ret5 = min5.close/min5.close.shift(1) - 1
ret5_un = ret5.reset_index()
ret5_un[ret5.name].groupby(ret5_un.datetime.dt.to_period("D"),group_keys=False).apply(lambda x: x.std())


datetime
2017-01-03    0.001236
2017-01-04    0.001911
2017-01-05    0.002549
2017-01-06    0.001624
2017-01-09    0.001272
                ...   
2017-12-25    0.006456
2017-12-26    0.003554
2017-12-27    0.002821
2017-12-28    0.004544
2017-12-29    0.003924
Freq: D, Name: close, Length: 239, dtype: float64

In [189]:
# from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

# x = load_cache('turn_Amihud_Pvc_2023_train', cache_type=CACHE_TYPE.FACTOR)
# x

turn_cv        amihud    amihud_adv  amihud_negative  \
date       code                                                            
2016-01-04 000001       NaN           NaN           NaN              NaN   
           000004       NaN           NaN           NaN              NaN   
           000006       NaN           NaN           NaN              NaN   
           000008       NaN           NaN           NaN              NaN   
           000009       NaN           NaN           NaN              NaN   
...                     ...           ...           ...              ...   
2020-12-31 605376       NaN  5.626010e-11  8.533268e-11              NaN   
           605377       NaN  6.775211e-11           NaN              NaN   
           605388  0.369998  5.593515e-11  2.062429e-10     1.291618e-10   
           605399  0.630154  5.437662e-10  4.370960e-10     3.455858e-10   
           605500       NaN  1.267159e-10  1.749631e-10              NaN   

                   amihud_cv           obv       pvc  pvc_optimize  
date       code                                                     
2016-01-04 000001        NaN           NaN       NaN           NaN  
           000004        NaN           NaN       NaN           NaN  
           000006        NaN           NaN       NaN           NaN  
           000008        NaN           NaN       NaN           NaN  
           000009        NaN           NaN       NaN           NaN  
...                      ...           ...       ...           ...  
2020-12-31 605376   1.387482  10895.735545 -0.191241     -0.557540  
           605377        NaN  -1604.340696       NaN           NaN  
           605388   4.518179  -1389.174595  0.586291     -0.055618  
           605399   4.903796    473.389921  0.801602      0.211291  
           605500   3.154291   -901.756496       NaN           NaN  

[3009425 rows x 8 columns]